In [3]:
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password

In [4]:
file = "SP500_from18_to20.csv"
df = pd.read_csv(file)
df.head()

,ticker,Date,High,Low,Open,Close,Volume,Adj Close
0,TJX,2018-02-08,38.404999,37.174999,38.270000,37.180000,10940000.0,35.848591
1,TJX,2018-02-09,38.125000,36.410000,37.294998,37.820000,15882000.0,36.465668
2,TJX,2018-02-12,38.189999,37.270000,38.125000,37.855000,10206800.0,36.499420
3,TJX,2018-02-13,38.095001,37.509998,37.755001,37.775002,6666600.0,36.422283
4,TJX,2018-02-14,38.465000,37.485001,37.544998,38.285000,8596600.0,37.067345


In [5]:
df = df.rename(columns = {'Name':'ticker',
                            'Date':'date_close',
                          'High':'high',
                          'Open':'open',
                          'Low':'low',
                          'Close':'close',
                          'Volume':'volume'
                           })
df = df[['ticker','date_close','high','open','low','close','volume']]
df_ticker = df[['ticker']]
df_ticker.drop_duplicates("ticker", inplace=True)
df.head()

<ipython-input-5-466dbaa3ae60>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ticker.drop_duplicates("ticker", inplace=True)


,ticker,date_close,high,open,low,close,volume
0,TJX,2018-02-08,38.404999,38.270000,37.174999,37.180000,10940000.0
1,TJX,2018-02-09,38.125000,37.294998,36.410000,37.820000,15882000.0
2,TJX,2018-02-12,38.189999,38.125000,37.270000,37.855000,10206800.0
3,TJX,2018-02-13,38.095001,37.755001,37.509998,37.775002,6666600.0
4,TJX,2018-02-14,38.465000,37.544998,37.485001,38.285000,8596600.0


In [6]:
connection_string = f'{username}:{password}@localhost:5432/Stock_DB'
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
engine.table_names()

['ticker_security', 'index_event', 'daily_data']

In [8]:
for i in range(len(df_ticker)):
    try:
        df_ticker.iloc[i:i+1].to_sql(name='ticker_security', con=engine, if_exists='append',index=False)
    except:
       pass #or any other action  

In [9]:
df.to_sql(name='daily_data', con=engine, if_exists='append', index=True)